In [1]:
from math import log2,comb as binom, log, sqrt
from random import randint, shuffle
import numpy as np
import hashlib
import struct
import random

In [2]:
def precalc_binoms():
    """
    precompute binomial coefficients
    """
    b=[[0 for j in range(w+1)] for i in range(n+1)]
    for i in range(n):
        for j in range(w+1):
            if i>=j:
                b[i][j]=binom(i,j)
    return b

In [3]:
def projection(a,n,w):
    """
    bijection between integers and length-n weigh-w binary strings
    """
    wt=w
    wn = n;
    wk = w;
    v = 0;
    setc = 0;
    while (wn != 0):
        
        if (setc == wt):
            break;
        elif (wn + setc == wt):
            v += (1 << (wn - 1));
            wn -= 1;
            setc += 1;
        elif (a < binom_list[wn - 1][wk]):
            wn -= 1;
        else:
            a -= binom_list[wn - 1][wk];
            v += (1 << (wn - 1));
            wn -= 1;
            wk -= 1;
            setc += 1;
    return v

In [4]:
def merge_vecs(v1,v2,n,w):
    """
    given n bit integer v1 with exactly n-w zeros and n-w bit integer v2 copy v2 to the zero
    positions of v1 with the 1s in v1 substituted by -1s
    """
    v1=((v1 & (1 << np.arange(n))) > 0).astype(np.int)
    v2=((v2 & (1 << np.arange(n-w))) > 0).astype(np.int)
    zero_indices = np.where(v1 == 0)[0]
    v2[v2==1]=-1
    v1[zero_indices] = v2
    return v1
            
def projection_ternary(a,n,w):
    """
    projection between integers and length-n ternary vectors with exactly w 1s and w -1s
    """
    B1 = binom(n,w)
    
    # split input in pair of numbers i1,i2 with ij<Bj
    i1=a%B1
    i2=int((a-i1)/B1)
    # determine positions of ones via projection
    v = projection(i1,n,w)

    # from remaing n-w not set entries determine positions of -1s (here 2s)
    v2 =projection(i2,n-w,w)

    # merge both choices in one vector
    s=merge_vecs(v,v2,n,w)
    
    return s

### Test the projections

In [5]:
n,w=12,2
binom_list=precalc_binoms()

amount = binom(n,w)*binom(n-w,w)
L=[]
for i in range(amount): 
    v=projection_ternary(i,n,w)
    L.append(v)

In [6]:
for i in range(len(L)):
    for j in range(len(L)):
        if i==j:
            continue
        elif all(L[i]==L[j]):
            print("error",i,j)
    

In [7]:
every_vector_has_desired_form=True
for i in L:
    if np.count_nonzero(i == 0)!= n-2*w or np.count_nonzero(i == 1)!=w or np.count_nonzero(i ==-1)!=w or len(i)!=n:
        every_vector_has_desired_form=False
if every_vector_has_desired_form and len(L)==amount:
    print("bijection works as intended")
else:
    print("ERROR: bijection not working")

bijection works as intended


# Collision Search stuff

In [8]:
def floyd(f, x0):
    """
    Floyds cycle finding returns colliding inputs and number of steps until collision was noticed
    """
    # Set up the initial parameters
    tortoise = f(x0)
    hare = f(f(x0))
    count = 0
    # Search for a collision
    while tortoise != hare:
        tortoise = f(tortoise)
        hare = f(f(hare))
        count+=1

    # Find the position of the first collision
    mu = 0
    tortoise = x0
    x1,x2=tortoise,hare
    while tortoise != hare:
        x1,x2=tortoise,hare
        tortoise = f(tortoise)
        hare = f(hare)
        mu += 1

    return x1,x2,count

In [9]:
def calc_lwe_func(s,switch=False):
    """
    calculates A*s or b-A*s for given vector s depending on switch
    """
    if not(switch):
        return A.dot(s)%q
    else:
        return (b-A.dot(s))%q
    
def calc_number(out):
    """
    convert vector in Zq^ell into log2(q)*ell bit integer
    """
    #only implemented for power of two till now for efficiency reason:
    a=out[0]
    for i in range(1,ell):
        a+=out[i]<<(i*bit_q)
    return a %domain


def flavour(a):
    """
    random permutation of a mod modul_p
    """
    return (f1*int(a)+f2)%modul_p

def collision_func(a):
    """
    function used for collision finding:
        - apply flavor
        - use bijection to obtain ternary vector v of desired weight
        - calculate A*v 
        - return the corresponding log2(q)*ell bit integer associated with the result
    """
    a=flavour(a)
    v = projection_ternary(a,n,w)
    out = calc_lwe_func(v,a%2)
    a= calc_number(out)
    return a 


# Random hash function that maps integers in the domain [0, M-1] to integers in the same domain using a cryptographic hash function
def random_function(x):
    # Convert x to bytes
    data = struct.pack('!I', flavour(x))
    
    # Apply a hash function to the combined data
    hash_obj = hashlib.sha256(data)
    digest_bytes = hash_obj.digest()

    # Convert the hash digest to an integer in the domain
    return int.from_bytes(digest_bytes, byteorder='big') % domain

    

### Set all necessary parameters

In [10]:
from sympy import prevprime

In [11]:
def exact_reps():
    """
    calculates the exact amount of representations (not only the symmetric case)
    """
    r=0
    for i in range(max(0,w_s-w),min(w_s,w)+1):
        r+=binom(w_s,i)**2*binom(n-2*w_s,w-i)*binom(n-2*w_s-(w-i),w-(w_s-i))
    return log2(r)

In [12]:

n=13
w=2
w_s=2
q=16
bit_q=int(log2(q))

binom_list=precalc_binoms()

if (q & (q - 1)) != 0:
    print("ERROR: currently only implemented for q being a power of two (for efficiency reasons)")
if w<w_s/2:
    print("ERROR: w needs to be at least w_s/2")
if w_s%2==1:
    print("ERROR: w_s must be even")

domain=binom(n,w)*binom(n-w,w)
ell_before=log(domain,q)
ell=int(round(ell_before))

if abs(ell-ell_before)>0.05:
    print("WARNING: currently the functions always match on full vector coordinates.")
    print("However, the necessary value of coordinates to match for this configuration")
    print("is",ell_before,"but it will be used", ell,"this will lead to deviations in the distributions.")
    print("you can change that by modifying n and w slightly")


#generate random secrect s with w_s 1s and w_s -1s
n_range=[i for i in range(n)]
shuffle(n_range)
s= np.array([0 for _ in range(n)])
for i in range(2*w_s):
    s[n_range[i]]=1 if i<w_s else -1
        
# generate A and corresponding b
A=np.array([[randint(0,q-1) for _ in range(n)] for _ in range(n)])
b = A.dot(s)%q


# amount of representations
reps=exact_reps()


# used for the flavor (the permutation)
f1=f2=0
modul_p= prevprime(domain)


In [13]:
print("Expected functions evals per collision", 2**(log2(domain)/2))

Expected functions evals per collision 65.49809157525128


In [14]:
expected_colls_until_solution=domain**2 /q**ell/2**(reps)
print("Expected collisions to find solution",expected_colls_until_solution)

Expected collisions to find solution 12.481079101562507


In [15]:
Lfunc_calls=[]
Lcolls=[]
found=0
total=0
#iteration stops ones the solution has been found that often
successes=200

while True:
    # choose new flavor
    global f1,f2
    f1,f2=randint(1,modul_p-1),randint(1,modul_p-1)
    
    # perform collision search
    x,y,lam=floyd(collision_func,randint(1,domain-1))
    
    # about half the collisions will be between the functions for the same switch.
    # those do not count towards the number of expected collisions, as they can not 
    # yield the solution.
    if flavour(x)%2!=flavour(y)%2:
        # Recompute vector given by collision
        v=(projection_ternary(flavour(x),n,w)+projection_ternary(flavour(y),n,w))
        bp=A.dot(v)%q

        # a few collisions will not match due to the not matching values of ell*log2(q) and log2(domain)
        # all for which the actual ell coordinates match are counted towards the found collisions
        if all(((b-bp)%q)[0:ell]==0):
            found+=1
            Lfunc_calls.append(lam)

        # if collision gives rise to solution store information 
        if all((projection_ternary(flavour(x),n,w)+projection_ternary(flavour(y),n,w))==s):
            Lcolls.append(found)
            found=0
            total+=1
            if total>=successes:
                break

In [16]:
sum(Lfunc_calls)/len(Lfunc_calls)

67.59991843393148

In [17]:
sum(Lcolls)/len(Lcolls)

12.26

### Perform collision search in random function

In [ ]:
Lrand=[]
for i in range(1000):
    global f1,f2
    f1,f2=randint(1,modul_p-1),randint(1,modul_p-1)
    x,y,lam=floyd(random_function,randint(1,domain-1))
    Lrand.append(lam)

In [19]:
print("Average of",sum(Lrand)/len(Lrand),

68.779